In [8]:
import numpy as np
import pandas as pd
import pymysql
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error
import pmdarima as pm
import os

In [9]:
user = os.environ.get('DB_USER')
password = os.environ.get('DB_PWD')
db = os.environ.get('DB_DB')
host = os.environ.get('DB_HOST')
connection = pymysql.connect(host=host, user = user, passwd = password, port = 3306, db = db)

In [10]:
df = pd.read_sql_query('SELECT incident_date, council_district, police_district, neighborhood FROM full_incidents',connection)

In [11]:
df = df[df.incident_date >= '2020-01-01']

In [12]:
connection.close()

In [13]:
prediction = pd.DataFrame(columns=['location', 'level', 'frequency', 'start_date', 'end_date', 'pred'])
performance = pd.DataFrame(columns=['entity', 'level', 'frequency', 'RMSE'])

In [14]:
def insert_into_df(label, level, index, FORECAST, rmse):
    global prediction, performance
    location = label
    level = level
    if len(index) == 365:
        frequency = 'DAILY'
    elif len(index) == 52:
        frequency = 'WEEKLY'
    elif len(index) == 12:
        frequency = 'MONTHLY'
    for i in range(len(index)):
        if frequency == 'DAILY':
            start_date = pd.to_datetime(index[i]).date()
            end_date = pd.to_datetime(index[i]).date()
        elif frequency == 'WEEKLY':
            start_date = pd.to_datetime(index[i]).date()-timedelta(days=7)
            end_date = pd.to_datetime(index[i]).date()
        elif frequency == 'MONTHLY':
            start_date = pd.to_datetime(index[i]).date()-timedelta(days=30)
            end_date = pd.to_datetime(index[i]).date()
        pred = FORECAST[i]
        prediction = prediction.append({'location':location, 'level':level, 'frequency':frequency, 'start_date':start_date, 'end_date':end_date, 'pred':pred }, ignore_index=True)   
    performance = performance.append({'entity':location, 'level':level, 'frequency':frequency, 'RMSE':rmse}, ignore_index=True)

In [15]:
def predict(data, index):
    MODEL = pm.auto_arima(data, seasonal = False, error_action="raise", stepwise=True, suppress_warnings=True, m=0)
    FORECAST = MODEL.predict(len(index))
    FORECAST = pd.Series(FORECAST, index = index)
    return FORECAST

In [16]:
def validate_model(data):
    data_train = data[:int(0.7*(len(data)))] 
    data_test = data[int(0.7*(len(data))):]
    model = pm.auto_arima(data_train, seasonal = False, error_action="raise", stepwise=True, suppress_warnings=True, m=0)
    forecast = model.predict(len(data_test))
    forecast = pd.Series(forecast, index = data_test.index)
    rmse = np.sqrt(mean_squared_error(data_test['no_of_incidents'].values, forecast.values))
    return rmse

In [17]:
def daily(data, label, level):
    rmse = validate_model(data)
    index = pd.date_range(data.index[-1]+timedelta(days=1), freq = 'D', periods = 365)
    FORECAST = predict(data, index)
    insert_into_df(label, level, index, FORECAST, rmse)

In [18]:
def weekly(data, label, level):
    data_w = data.resample('W').sum()
    rmse_w = validate_model(data_w)
    index_w = pd.date_range(data_w.index[-1]+timedelta(days=7), freq = 'W', periods = 52)
    FORECAST_w = predict(data_w, index_w)
    insert_into_df(label, level, index_w, FORECAST_w, rmse_w)

In [19]:
def monthly(data, label, level):
    data_m = data.resample('M').sum()
    rmse_m = validate_model(data_m)
    index_m = pd.date_range(data_m.index[-1]+timedelta(days=30), freq = 'M', periods = 12)
    FORECAST_m = predict(data_m, index_m)
    insert_into_df(label, level, index_m, FORECAST_m, rmse_m)

In [20]:
levels = ['council_district', 'police_district', 'neighborhood']
for i in range(3):
    df_level = df.iloc[:,[0,i+1]]
    df_level = pd.DataFrame({'no_of_incidents' : df_level.groupby( [ df[levels[i]], df_level['incident_date'].dt.date] ).size()}).reset_index()
    df_level = df_level.set_index('incident_date')
    df_level = df_level[df_level[levels[i]] != '']
    df_level = df_level[df_level[levels[i]] != 'UNKNOWN']
    labels = df_level[levels[i]].unique()
    for l in labels:
        label_data = df_level[df_level[levels[i]]==l]
        label_data = label_data[['no_of_incidents']]
        label_data = label_data.asfreq('D')
        label_data.no_of_incidents= label_data.no_of_incidents.fillna(0.0)
        #Some neighborhood codes throwing errors, hence excluding them 
        try:
            daily(label_data, l, levels[i])
            weekly(label_data, l, levels[i])
            monthly(label_data, l, levels[i])
        except:
            continue

C:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1890: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
C:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1897: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
C:\Users\HP\AppData\Roaming\Python\Python38\site-packages\pmdarima\arima\auto.py:460: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
C:\Users\HP\AppData\Roaming\Python\Python38\site-packages\pmdarima\arima\auto.py:460: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
C:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1890: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots

In [21]:
prediction

,location,level,frequency,start_date,end_date,pred
0,Allentown,council_district,DAILY,2021-03-05,2021-03-05,0.219661
1,Allentown,council_district,DAILY,2021-03-06,2021-03-06,0.144558
2,Allentown,council_district,DAILY,2021-03-07,2021-03-07,0.238797
3,Allentown,council_district,DAILY,2021-03-08,2021-03-08,0.170760
4,Allentown,council_district,DAILY,2021-03-09,2021-03-09,0.292042
...,...,...,...,...,...,...
65953,6008,neighborhood,MONTHLY,2021-10-01,2021-10-31,0.800000
65954,6008,neighborhood,MONTHLY,2021-10-31,2021-11-30,0.800000
65955,6008,neighborhood,MONTHLY,2021-12-01,2021-12-31,0.800000
65956,6008,neighborhood,MONTHLY,2022-01-01,2022-01-31,0.800000


In [22]:
performance

,entity,level,frequency,RMSE
0,Allentown,council_district,DAILY,0.417468
1,Allentown,council_district,WEEKLY,1.136684
2,Allentown,council_district,MONTHLY,3.287856
3,Black Rock,council_district,DAILY,0.294575
4,Black Rock,council_district,WEEKLY,0.727813
...,...,...,...,...
448,6007,neighborhood,WEEKLY,0.420687
449,6007,neighborhood,MONTHLY,0.974130
450,6008,neighborhood,DAILY,0.162373
451,6008,neighborhood,WEEKLY,0.381881


In [23]:
connection = pymysql.connect(host=host, user = user, passwd = password, port = 3306, db = db)
cur = connection.cursor()

In [24]:
cur.execute("TRUNCATE TABLE predictions")
cur.execute("TRUNCATE TABLE performance")
connection.commit()

In [25]:
for (row,rs) in prediction.iterrows():
    rs[3] = rs[3].strftime('%Y-%m-%d')
    rs[4] = rs[4].strftime('%Y-%m-%d')
    rs[5] = str(rs[5])
    cur.execute("INSERT INTO predictions(location,level,freq,start_date,end_date,predicted_incidents)VALUES('"+rs[0]+"', '"+rs[1]+"', '"+rs[2]+"', '"+rs[3]+"', '"+rs[4]+"', "+rs[5]+")")

OperationalError: (1054, "Unknown column 'freq' in 'field list'")

In [26]:
for (row,rs) in performance.iterrows():
    rs[3] = str(rs[3])
    cur.execute("INSERT INTO performance(entity,level,freq,rmse)VALUES('"+rs[0]+"', '"+rs[1]+"', '"+rs[2]+"', "+rs[3]+")")   

In [27]:
connection.commit()

In [28]:
df = pd.read_sql_query('SELECT * FROM predictions',connection)
df

,id,location,level,start_date,end_date,predicted_incidents,lower_predicted_incidents,upper_predicted_incidents


In [29]:
df = pd.read_sql_query('SELECT * FROM performance',connection)
df

,id,entity,level,freq,rmse
0,1,Allentown,council_district,0000-00-00,0.417468
1,2,Allentown,council_district,0000-00-00,1.136684
2,3,Allentown,council_district,0000-00-00,3.287856
3,4,Black Rock,council_district,0000-00-00,0.294575
4,5,Black Rock,council_district,0000-00-00,0.727813
...,...,...,...,...,...
448,449,6007,neighborhood,0000-00-00,0.420687
449,450,6007,neighborhood,0000-00-00,0.974130
450,451,6008,neighborhood,0000-00-00,0.162373
451,452,6008,neighborhood,0000-00-00,0.381881


In [30]:
connection.close()